In [3]:
from nltk.stem import LancasterStemmer
import nltk
import numpy as np
import random
import json
import pickle

In [2]:
with open("intents.json") as f:
    data=json.load(f)

In [34]:
words,labels=[],[]
x_docs,y_docs=[],[]

for intents in data["intents"]:
    for patterns in intents["patterns"]:
        wrds=nltk.word_tokenize(patterns)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intents["tag"])

    if intents["tag"] not in labels:
        labels.append(intents["tag"])

In [35]:
stemmer=LancasterStemmer()
words=[stemmer.stem(w.lower()) for w in words if w!="?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [36]:
training,output=[],[]
output_data=[0 for i in range(len(labels))]

for i,text in enumerate(x_docs):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in text]
    for w in words:
        if w in wrds:
            bag.append(1)

        else:
            bag.append(0)

    output_row=output_data[:]
    output_row[labels.index(y_docs[i])]=1

    training.append(bag)
    output.append(output_row)

In [37]:
training=np.array(training)
output=np.array(output)

In [38]:
output.shape

(188, 44)

In [39]:
training[1].shape

(114,)

In [40]:
from keras.models import Sequential
from keras.layers import Dense

In [44]:
model=Sequential()
model.add(Dense(units=100,activation="relu",input_shape=training[1].shape))
model.add(Dense(units=50,activation="relu"))
model.add(Dense(units=50,activation="relu"))
model.add(Dense(units=44,activation="softmax"))

In [45]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(training,output,epochs=20,batch_size=8)

Train on 188 samples
Epoch 1/20
188/188 [==============================] - 0s 532us/sample - loss: 3.7929 - acc: 0.0266
Epoch 2/20
188/188 [==============================] - 0s 170us/sample - loss: 3.7258 - acc: 0.0638
Epoch 3/20
188/188 [==============================] - 0s 186us/sample - loss: 3.6545 - acc: 0.1489
Epoch 4/20
188/188 [==============================] - 0s 175us/sample - loss: 3.5397 - acc: 0.2181
Epoch 5/20
188/188 [==============================] - 0s 165us/sample - loss: 3.3621 - acc: 0.2234
Epoch 6/20
188/188 [==============================] - 0s 186us/sample - loss: 3.0569 - acc: 0.3777
Epoch 7/20
188/188 [==============================] - 0s 186us/sample - loss: 2.6069 - acc: 0.5000
Epoch 8/20
188/188 [==============================] - 0s 181us/sample - loss: 2.0314 - acc: 0.7074
Epoch 9/20
188/188 [==============================] - 0s 198us/sample - loss: 1.4625 - acc: 0.8883
Epoch 10/20
188/188 [==============================] - 0s 181us/sample - loss: 0.9675 - 

In [46]:
with open("data.pickle","wb") as f:
    pickle.dump((words,labels,training,output),f)

In [47]:
model.save("aid_chatbot.h5")

In [49]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(w.lower()) for w in s_words]

    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1

    return np.array(bag)

In [55]:
s="Please help me i have fever"

bag=bag_of_words(s,words)
bag=np.reshape(bag,(1,bag.shape[0]))
pred=model.predict([bag])
tag=labels[np.argmax(pred)]

for tg in data["intents"]:
    if tg["tag"]==tag:
        response=tg["responses"]

random.choice(response)

'To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.'